In [0]:
!curl http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Cell_Phones_and_Accessories_5.json.gz -o reviews.json.gz
!gunzip reviews.json.gz
!ls

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  161M  100  161M    0     0  10.3M      0  0:00:15  0:00:15 --:--:-- 10.7M
reviews.json  sample_data


In [0]:
import pandas as pd


In [0]:
df=pd.read_json('reviews.json', lines=True)

In [0]:
df.head()

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,5,True,"08 4, 2014",A24E3SXTC62LJI,7508492919,{'Color:': ' Bling'},Claudia Valdivia,Looks even better in person. Be careful to not...,Can't stop won't stop looking at it,1407110400,NaN,NaN
1,5,True,"02 12, 2014",A269FLZCB4GIPV,7508492919,NaN,sarah ponce,When you don't want to spend a whole lot of ca...,1,1392163200,NaN,NaN
2,3,True,"02 8, 2014",AB6CHQWHZW4TV,7508492919,NaN,Kai,"so the case came on time, i love the design. I...",Its okay,1391817600,NaN,NaN
3,2,True,"02 4, 2014",A1M117A53LEI8,7508492919,NaN,Sharon Williams,DON'T CARE FOR IT. GAVE IT AS A GIFT AND THEY...,CASE,1391472000,NaN,NaN
4,4,True,"02 3, 2014",A272DUT8M88ZS8,7508492919,NaN,Bella Rodriguez,"I liked it because it was cute, but the studs ...",Cute!,1391385600,NaN,NaN


In [0]:
df.describe()

,overall,unixReviewTime
count,1.128437e+06,1.128437e+06
mean,4.221383e+00,1.440400e+09
std,1.231917e+00,4.569215e+07
min,1.000000e+00,1.035331e+09
25%,4.000000e+00,1.416355e+09
50%,5.000000e+00,1.444349e+09
75%,5.000000e+00,1.470442e+09
max,5.000000e+00,1.538438e+09


In [0]:
#drop extra columns
df1=df.drop(columns=['vote', 'image','style'])

In [0]:
#combine the two columns to make one
df1["review"] = df1["reviewText"] + df1["summary"]

In [0]:
#drop repeated columns
df2=df1.drop(columns=['reviewText', 'summary'])

In [0]:
df2.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,unixReviewTime,review
0,5,True,"08 4, 2014",A24E3SXTC62LJI,7508492919,Claudia Valdivia,1407110400,Looks even better in person. Be careful to not...
1,5,True,"02 12, 2014",A269FLZCB4GIPV,7508492919,sarah ponce,1392163200,When you don't want to spend a whole lot of ca...
2,3,True,"02 8, 2014",AB6CHQWHZW4TV,7508492919,Kai,1391817600,"so the case came on time, i love the design. I..."
3,2,True,"02 4, 2014",A1M117A53LEI8,7508492919,Sharon Williams,1391472000,DON'T CARE FOR IT. GAVE IT AS A GIFT AND THEY...
4,4,True,"02 3, 2014",A272DUT8M88ZS8,7508492919,Bella Rodriguez,1391385600,"I liked it because it was cute, but the studs ..."


In [0]:
from sklearn.model_selection import StratifiedShuffleSplit
print("Before {}".format(len(df2)))
df2After = df2.dropna(subset=["overall"]) # removes all NAN in reviews.rating
print("After {}".format(len(df2After)))
df2After["overall"] = df2After["overall"].astype(int)

Before 1128437
After 1128437


In [0]:
split = StratifiedShuffleSplit(n_splits=5, test_size=0.2)
for train_index, test_index in split.split(df2After, df2After["overall"]): 
    strat_train = df2After.reindex(train_index)
    strat_test = df2After.reindex(test_index)


In [0]:
def sentiments(rating):
    if (rating == 5) or (rating == 4):
        return "Positive"
    elif rating == 3:
        return "Neutral"
    elif (rating == 2) or (rating == 1):
        return "Negative"
# Add sentiments to the data
strat_train["Sentiment"] = strat_train["overall"].apply(sentiments)
strat_test["Sentiment"] = strat_test["overall"].apply(sentiments)
strat_train["Sentiment"][:20]

1028917     Neutral
734279     Positive
1120962     Neutral
718137     Positive
399041     Positive
719452     Positive
976436     Positive
934826      Neutral
186972      Neutral
1108039     Neutral
325630     Negative
246880     Positive
355377     Positive
1034050    Positive
617208     Positive
618146     Positive
136039     Negative
978507     Positive
836466     Positive
686983     Positive
Name: Sentiment, dtype: object

Start to train:

In [0]:
# Prepare data
X_train = strat_train["review"]
X_train_targetSentiment = strat_train["Sentiment"]
X_test = strat_test["review"]
X_test_targetSentiment = strat_test["Sentiment"]
print(len(X_train), len(X_test))

902749 225688


In [0]:
# Replace "nan" with space
X_train = X_train.fillna(' ')
X_test = X_test.fillna(' ')
X_train_targetSentiment = X_train_targetSentiment.fillna(' ')
X_test_targetSentiment = X_test_targetSentiment.fillna(' ')

# Text preprocessing and occurance counting
from sklearn.feature_extraction.text import CountVectorizer 
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train) 
X_train_counts.shape

(902749, 182385)

In [0]:
from sklearn.naive_bayes import MultinomialNB

In [0]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer(use_idf=False)
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape


(902749, 182385)

In [0]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
multiNB_pipe = Pipeline([("vect", CountVectorizer()), ("tfidf", TfidfTransformer()), ("clf_nominalNB", MultinomialNB())])
multiNB_pipe.fit(X_train, X_train_targetSentiment)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf_nominalNB',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [0]:
from sklearn.metrics import accuracy_score
import numpy as np
predictedMultiNB = multiNB_pipe.predict(X_test)
#np.mean(predictedMultiNB == X_test_targetSentiment)
accuracy_score(predictedMultiNB, X_test_targetSentiment)

0.8298358796214242

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
logReg_pipe = Pipeline([("vect", CountVectorizer()),("tfidf", TfidfTransformer()), ("clf_logReg", LogisticRegression())])
logReg_pipe.fit(X_train, X_train_targetSentiment)

import numpy as np
predictedLogReg = logReg_pipe.predict(X_test)
#np.mean(predictedLogReg == X_test_targetSentiment)
accuracy_score(predictedLogReg, X_test_targetSentiment)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.8965430151359399

In [0]:
from sklearn.svm import LinearSVC
linearSVC_pipe = Pipeline([("vect", CountVectorizer()), ("tfidf", TfidfTransformer()), ("clf_linearSVC", LinearSVC())])
linearSVC_pipe.fit(X_train, X_train_targetSentiment)

predictedLinearSVC = linearSVC_pipe.predict(X_test)
#np.mean(predictedLinearSVC == X_test_targetSentiment)
accuracy_score(predictedLinearSVC, X_test_targetSentiment)

0.899883910531353

In [0]:
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)],    
             'tfidf__use_idf': (True, False), 
             } 
gs_LinearSVC_pipe = GridSearchCV(linearSVC_pipe, parameters, n_jobs=-1)
gs_LinearSVC_pipe.fit(X_train, X_train_targetSentiment)
predictedGS_LinearSVC_pipe = gs_LinearSVC_pipe.predict(X_test)
accuracy_score(predictedGS_LinearSVC_pipe, X_test_targetSentiment)

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


0.9145678990464713

In [0]:
gs_LinearSVC_pipe.best_params_

{'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}